In [151]:
# The datasets package needs to be loaded to access our data 
library(datasets)
dataset = iris
head(dataset)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


In [152]:
#Encoding the Species as a factor
dataset$Species = factor(dataset$Species, 
                         levels = c("setosa", "versicolor", "virginica"))
                            
head(dataset)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


In [153]:

# Splitting the dataset into the Training set and Test set

library(caTools)
set.seed(123)
split = sample.split(dataset$Species, SplitRatio = 0.75)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)


In [154]:
cat("Training size: ", nrow(training_set), "\nTest size: ", nrow(test_set))

Training size:  114 
Test size:  36

In [155]:
head(training_set)
head(test_set)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
9,4.4,2.9,1.4,0.2,setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
8,5.0,3.4,1.5,0.2,setosa
11,5.4,3.7,1.5,0.2,setosa
16,5.7,4.4,1.5,0.4,setosa
20,5.1,3.8,1.5,0.3,setosa


In [170]:
#scatter plot of training set
library(dplyr)
library(ggplot2)

training_set %>% ggplot(aes(x = Sepal.Length, y = Sepal.Width, color = Species)) +
   geom_point(size = 3)





In [171]:
training_set %>% ggplot(aes(x = Petal.Length, y = Petal.Width, color = Species)) +
   geom_point(size = 3)


In [172]:
#scatter plot of test set
test_set %>% ggplot(aes(x = Sepal.Length, y = Sepal.Width, color = Species)) +
   geom_point(size = 3)

In [157]:
# Fitting K-NN to the Training set and Predicting the Test set results
library(class)
y_pred = knn(train = training_set[,-5],
             test = test_set[,-5],
             k = 5,
             cl = training_set[,5], 
             prob = TRUE)

# Making the Confusion Matrix
cm = table(test_set[, 5], y_pred)
cm

            y_pred
             setosa versicolor virginica
  setosa         12          0         0
  versicolor      0         10         2
  virginica       0          1        11

In [158]:
#Graph of error test in training set


library(class)
error.train = replicate(0,30)

for(k in 1:30) 
{
    pred_y = knn(train = training_set[,-5], test = training_set[,-5], k,
                     cl = training_set[,5])
    diff = mean(as.numeric(pred_y) == as.numeric(training_set[,5]))
    error.train[k] = 1 - diff
}

error.train <- unlist(error.train, use.names=FALSE)
plot(error.train, type="o", 
     col="blue", xlab = "K values", ylab = "Misclassification errors")


In [159]:
#Graph of error test in test set


library(class)
error.test = replicate(0,30)

for(k in 1:30) 
{
    pred_y = knn(train = training_set[,-5], test = test_set[,-5], k,
                     cl = training_set[,5])
    diff = mean(as.numeric(pred_y) == as.numeric(test_set[,5]))
    error.test[k] = 1 - diff
}

error.test <- unlist(error.test, use.names=FALSE)


plot(error.train, type="o", ylim=c(0,0.15), col="blue", xlab = "K values", ylab = "Misclassification errors")
lines(error.test, type = "o", col="red")
legend("topright", legend=c("Training error","Test error"), col = c("blue","red"), lty=1:1)

In [178]:
#Use k = 4 for best result

pred_iris = knn(train = training_set[,-5], test = test_set[,-5], 
                k = 4, cl = training_set[,5])

cm = table(test_set[, 5], pred_iris)
cm
result = cbind(test_set, pred_iris)



            pred_iris
             setosa versicolor virginica
  setosa         12          0         0
  versicolor      0         11         1
  virginica       0          1        11

In [174]:
head(result)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,pred_iris
4,4.6,3.1,1.5,0.2,setosa,setosa
5,5.0,3.6,1.4,0.2,setosa,setosa
8,5.0,3.4,1.5,0.2,setosa,setosa
11,5.4,3.7,1.5,0.2,setosa,setosa
16,5.7,4.4,1.5,0.4,setosa,setosa
20,5.1,3.8,1.5,0.3,setosa,setosa


In [176]:
# Graph of the result
result%>%
    ggplot(aes(x=Petal.Width, y=Petal.Length, color=pred_iris))+
    geom_point(size=3)

In [177]:
# Graph of the acutal 

result %>%
    ggplot(aes(x=Petal.Width, y=Petal.Length, color=Species))+
    geom_point(size=3)